In [1]:
!pip install scikit-plot
!pip install ktrain

     |████████████████████████████████| 92kB 12.3MB/s 
     |████████████████████████████████| 1.0MB 41.3MB/s 
     |████████████████████████████████| 245kB 52.5MB/s 
     |████████████████████████████████| 1.8MB 61.8MB/s 
  Created wheel for ktrain: filename=ktrain-0.7.2-cp36-none-any.whl size=113492 sha256=9989c2e0512e0c816ff625b5f403d0ef5ae03be7f85e3f15ddcf3b50c695b0cc
  Stored in directory: /root/.cache/pip/wheels/ed/db/bd/fb2cb12563e8a7e5ef04f99cc9014bc0e509f1bb3834e6ee5d
  Created wheel for keras-bert: filename=keras_bert-0.80.0-cp36-none-any.whl size=37923 sha256=82322a8f53d765a3cabd9f44e5963254b45d019e7f0c6e882a152ef79570c8c2
  Stored in directory: /root/.cache/pip/wheels/63/dc/87/3260cb91f3aa32c0f85c5375429a30c8fd988bbb48f5ee21b0
  Created wheel for langdetect: filename=langdetect-1.0.7-cp36-none-any.whl size=993460 sha256=c02ea28294984b047d4d9276c1e77df477048a884712c043e2d777de39ba6f95
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e3328

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import re
import numpy as np

import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier

import gensim

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import scikitplot as skplt
from sklearn.preprocessing import LabelEncoder

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from collections import Counter

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

import ktrain
from ktrain import text

Using TensorFlow backend.


using Keras version: 2.2.4-tf


In [0]:
nltk.download('stopwords')
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
#df = pd.read_csv('training.csv', encoding ='ISO-8859-1' , names=["target", "ids", "date", "flag", "user", "text"])
df = pd.read_csv('data.csv')

df.head()

,target,text
0,Negativo,aww bummer shoulda got david carr third day
1,Negativo,upset updat facebook text might cri result sch...
2,Negativo,dive mani time ball manag save rest go bound
3,Negativo,whole bodi feel itchi like fire
4,Negativo,behav mad see


In [0]:
len(df)

1592023

In [0]:
def drop_repeated(token):
    token = re.sub(r'(.)\1+', r'\1\1', token)     
    token = re.sub('(a*(?:ha)+h?){2,}', 'haha', token)
    token = re.sub('((?:l+o+)+l+){2,}', 'lol', token)
    token = re.sub('(n*(?:no)+o?n?){2, }', 'no', token)
    
    return token

In [0]:
def preprocess(text, stem=False):
    text = re.sub('@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+|\d', ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        token_norepeat = drop_repeated(token)
        stem_token = stemmer.stem(token_norepeat)
        if token not in stop_words and stem_token not in tokens:
            tokens.append(stem_token)
    return " ".join(tokens)

In [0]:
df.text = df.text.apply(lambda x: preprocess(x))

In [0]:
text_positive = " ".join(tweet for tweet in df.text[df.target=='Positivo'])
text_negative = " ".join(tweet for tweet in df.text[df.target=='Negativo'])

In [0]:
%%time

# Create and generate a word cloud image:
wordcloud = WordCloud(max_font_size=100, max_words=1000, background_color="white").generate(text_positive)
plt.figure(figsize=(10,10))
# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')

plt.axis("off")
plt.show()

In [0]:
%%time

# Create and generate a word cloud image:
wordcloud = WordCloud(max_font_size=100, max_words=1000, background_color="white").generate(text_negative)
plt.figure(figsize=(10,10))
# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')

plt.axis("off")
plt.show()

## Naive bayes

In [0]:
def word_feats(row):
    words = row['text']
    return dict([(word, True) for word in words.split()]), row['target']

In [0]:
features = df.apply(lambda x: word_feats(x), axis=1)

In [0]:
df_train, df_test = train_test_split(features, test_size=1-0.8)
print("Train size:", len(df_train))
print("Test size:", len(df_test))

In [0]:
classifier = NaiveBayesClassifier.train(df_train)

In [0]:
df_test_feat = [x[0] for x in df_test]
df_test_labels = [x[1] for x in df_test]

In [0]:
predictions = classifier.classify_many(df_test_feat)

In [0]:
print('Accuracy: '+ str(nltk.classify.util.accuracy(classifier, df_test)))
classifier.show_most_informative_features()

In [0]:
skplt.metrics.plot_confusion_matrix(
    df_test_labels, 
    predictions,
    figsize=(12,12))

## Logistic regression

In [0]:
cv=CountVectorizer()
word_count_vector=cv.fit_transform(df.text)
word_count_vector.shape

In [0]:
sample_index = np.random.random(df.shape[0])
X_train, X_test = word_count_vector[sample_index <= 0.8, :], word_count_vector[sample_index >  0.8, :]
Y_train, Y_test = df.target[sample_index <=  0.8], df.target[sample_index >  0.8]

In [0]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

In [0]:
tf_idf_vector_train = tfidf_transformer.transform(X_train)
tf_idf_vector_test = tfidf_transformer.transform(X_test)

In [0]:
feature_names = cv.get_feature_names()
 
first_document_vector=tf_idf_vector_train[0]
 
df_tfidf = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df_tfidf.sort_values(by=["tfidf"],ascending=False)

In [0]:
Logistic_model = LogisticRegression()

In [0]:
Logistic_model.fit(X_train, Y_train)

In [0]:
predictions = Logistic_model.predict(X_test)
accuracy_score(Y_test, predictions)

In [0]:
skplt.metrics.plot_confusion_matrix(
    Y_test, 
    predictions,
    figsize=(12,12))

## Bert

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data = pd.read_csv('/content/drive/My Drive/Twitter/data.csv')
data['positive'] = (data.target=='Positivo').astype(int)
data['negative'] = (data.target!='Positivo').astype(int)
data.to_csv('data_numeric.csv', index=False)

In [6]:
data_numeric = pd.read_csv('data_numeric.csv')
data_numeric.head()

,target,text,positive,negative
0,Negativo,aww bummer shoulda got david carr third day,0,1
1,Negativo,upset updat facebook text might cri result sch...,0,1
2,Negativo,dive mani time ball manag save rest go bound,0,1
3,Negativo,whole bodi feel itchi like fire,0,1
4,Negativo,behav mad see,0,1


In [7]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_csv('data_numeric.csv',
                    'text',
                    label_columns = ["positive", "negative"],
                    maxlen=20,
                    max_features=500,
                    ngram_range=2,
                    val_pct=0.7,
                    preprocess_mode='bert')

detected encoding: utf-8 (if wrong, set manually)
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


preprocessing test...
language: en


In [0]:
model = text.text_classifier('bert', (x_train, y_train), preproc=preproc)

learner = ktrain.get_learner(model, 
                             train_data=(x_train, y_train), 
                             val_data=(x_test, y_test), 
                             batch_size=128)

Is Multi-Label? False
maxlen is 20
done.


In [0]:
learner.fit(0.0001, 1, checkpoint_folder='/content/drive/My Drive/Twitter/checkpoints')

Train on 477606 samples, validate on 1114417 samples
477606/477606 [==============================] - 2238s 5ms/sample - loss: 0.4798 - acc: 0.7687 - val_loss: 0.4602 - val_acc: 0.7840


In [12]:
!ls '/content/drive/My Drive/Twitter/bertModel/model'

'/content/drive/My Drive/Twitter/bertModel/model'


In [0]:
learner.save_model('/content/drive/My Drive/Twitter/bertModel/model')

In [4]:
predictor = ktrain.load_predictor('/content/drive/My Drive/Twitter/bertModel/model')

load_predictor failed.
Could not find the saved preprocessor (/content/drive/My Drive/Twitter/bertModel/model.preproc) for this model. Are you sure predictor.save method was called?
